
**[PT]** Para executar este ficheiro veja o ficheiro README.md nesta directoria

**[EN]** To run this file: follow instructions in the README.md file in this directory.



# **[PT]** Atributos registados no nome
# **[EN]** Name attributes


**[EN]** Student names can have annotations in the name itself, between parentesis:

* D. (dom),  prefix for noble people, also Barão, "Visconte", "Marquês"
* Padre = Priest
* Frei  = Friar, normally followed by the name of the Order, with many variations
* Colegial ou Colégio = The college of the student, followed by the name of the college.
* Lente = Lecturer
* various professions: impressor, ourives, "mestre" 

In this notebook some of those attributes are analysed.

## Setup

In [3]:
from timelinknb import current_time,current_machine, get_mhk_db
import ucalumni.config as alumniconf

db_name = alumniconf.mhk_db_name
db = get_mhk_db(db_name)
print(current_machine,current_time,f'db={db_name}')

mini-m1.local 2022-05-11 04:19:58.548265 db=ucalumni


## **[PT]** Notas ao nome do estudante
## **[EN]** Notes to the student name

Get all the students with notes in the name.

In [4]:
from timelinknb.pandas import attribute_to_df


# Get list of people with attribute nome-nota and other derived attributes
name_notes = attribute_to_df(
                    the_type='nome-nota',
                    person_info=True,
                    more_cols=['uc-entrada','uc-saida','ordem-religiosa','titulo','padre', 'colegio'],
                    sql_echo=False)
name_notes.drop(['nome-nota.date','nome-nota.obs',
                 'ordem-religiosa.date','ordem-religiosa.obs',
                'titulo.date','titulo.obs',
                'padre.date','padre.obs',
                'colegio.date','colegio.obs',
                'uc-entrada.date','uc-entrada.obs',
                'uc-saida.date','uc-saida.obs'],axis=1, inplace=True)

print("Number of records with name notes:",len(name_notes.index.unique()))
print()
name_notes.info()

Number of records with name notes: 10044

<class 'pandas.core.frame.DataFrame'>
Index: 10416 entries, 127770 to 356441
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             10416 non-null  object
 1   sex              10416 non-null  object
 2   nome-nota        10416 non-null  object
 3   uc-entrada       10416 non-null  object
 4   uc-saida         10416 non-null  object
 5   ordem-religiosa  3437 non-null   object
 6   titulo           4219 non-null   object
 7   padre            7138 non-null   object
 8   colegio          379 non-null    object
dtypes: object(9)
memory usage: 1.0+ MB


In [5]:
name_notes.head(10).fillna("")

,name,sex,nome-nota,uc-entrada,uc-saida,ordem-religiosa,titulo,padre,colegio
id,,,,,,,,,
127770,José Cabaço,m,padre,1722-11-12,1738-05-04,,,sim,
127785,Baltasar Dias Cabeçudo,m,padre,1688-10-01,1691-10-01,,,sim,
127802,Francisco João do Cabo,m,padre,1693-10-01,1693-10-01,,,sim,
127813,Aires Pires Cabral,m,colégio de São Pedro,1559-11-01,1577-07-31,,,,Colégio de S.Pedro
127987,António Cabral,m,frei religioso de São Bernardo,1665-10-15,1666-10-10,Ordem de São Bernardo,frei,,
127997,António Cabral,m,padre,1684-10-01,1694-07-07,,,sim,
128013,António Cabral,m,padre,1642-10-29,1651-05-21,,,sim,
128016,António Cabral,m,padre,1639-10-21,1639-10-21,,,sim,
128122,Bernardo José Osório Cabral,m,padre,1744-10-30,1749-10-01,,,sim,


## **[EN]** Most frequent notes

In [6]:
import numpy as np

no_crossref = name_notes['uc-entrada'] != '0000-00-00'
notes_ncf = name_notes[no_crossref]
notes_count = notes_ncf.groupby('nome-nota')['uc-entrada'].agg(['count','min','max'])
notes_count.sort_values('count',ascending = False, inplace=True)
notes_count.head(10)


,count,min,max
nome-nota,,,
padre,5714,1537-00-00,1899-10-02
D.,600,1537-00-00,1910-10-20
frei monge de São Bento,174,1728-10-01,1799-10-29
frei monge de São Bernardo,154,1732-10-01,1827-10-29
padre frei religioso de Nossa Senhora do Carmo,96,1664-10-01,1760-10-01
padre frei religioso de São Bernardo,93,1664-10-01,1751-07-17
frei monge de São Jerónimo,90,1729-11-07,1825-10-25
padre frei religioso de Nossa Senhora da Graça,85,1669-10-01,1759-10-01
frei,71,1537-12-28,1830-10-22


In [7]:
notes_count.to_csv('../inferences/name-notes/note_originals.csv',sep=',')

### **[EN]** Priests

In [8]:
## **[EN]** Priests

priests = attribute_to_df(
                    the_type='padre',
                    the_value='sim',
                    person_info=True,
                    more_cols=['uc-entrada','uc-saida','naturalidade','nome-pai','ordem-religiosa','faculdade','titulo','colegio','nome-nota'],
                    sql_echo=False)
priests.drop(['nome-nota.date','nome-nota.obs','padre.date','padre.obs',
                 'ordem-religiosa.date','ordem-religiosa.obs','sex',
                 'naturalidade.date','naturalidade.obs',
                 'nome-pai.date','nome-pai.obs',
                 'faculdade.date','faculdade.obs',
                'titulo.date','titulo.obs',
                'colegio.date','colegio.obs',
                'uc-entrada.date','uc-entrada.obs',
                'uc-saida.date','uc-saida.obs'],axis=1, inplace=True)
priests = priests[priests['uc-entrada'] != '0000-00-00']
priests = priests[~priests.index.duplicated(keep='first')]
priests.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6916 entries, 127770 to 356441
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             6916 non-null   object
 1   padre            6916 non-null   object
 2   uc-entrada       6916 non-null   object
 3   uc-saida         6916 non-null   object
 4   naturalidade     5482 non-null   object
 5   nome-pai         2681 non-null   object
 6   ordem-religiosa  1153 non-null   object
 7   faculdade        6891 non-null   object
 8   titulo           1133 non-null   object
 9   colegio          41 non-null     object
 10  nome-nota        6916 non-null   object
dtypes: object(11)
memory usage: 648.4+ KB


In [9]:
priests_with_naturalidade = priests['naturalidade'].notnull()
priests_with_faculdade = priests['faculdade'].notnull()
priests_with_order = priests['ordem-religiosa'].notnull()
priests_with_father = priests['nome-pai'].notnull()
priests_with_title = priests['titulo'].notnull()
priests_with_colegio = priests['colegio'].notnull()
priests_full_info = priests_with_faculdade & priests_with_order & priests_with_colegio
priests[priests_with_title&priests_with_father | priests_with_colegio].sort_values('uc-entrada').head().fillna("")

,name,padre,uc-entrada,uc-saida,naturalidade,nome-pai,ordem-religiosa,faculdade,titulo,colegio,nome-nota
id,,,,,,,,,,,
150020,Manuel Batista,sim,1564-10-01,1568-06-21,,,,Teologia,,Colégio de S.João Evangelista,padre do colégio dos Lóios
193400,Jorge,sim,1564-10-01,1565-06-06,,,Ordem de Cristo,Teologia,frei,Colégio de Tomar,padre frei do colégio de Cristo
205601,António da Trindade,sim,1578-06-30,1588-04-02,,,Ordem da Santíssima Trindade,Teologia,frei,Colégio da Trindade,padre frei do colégio da Trindade
201701,João da Costa,sim,1580-03-22,1590-01-05,Lisboa,Fernão Nunes da Costa,,Cânones,cónego,,padre cónego na Sé de Coimbra
193431,Aleixo Jorge,sim,1582-10-22,1594-06-11,Vila Boa,Jorge Gonçalves,,Teologia,capelão,,padre capelão da Universidade


In [10]:
priests.sort_values(['name','uc-entrada']).to_csv('../inferences/name-notes/priests.csv',sep=',')

### **[EN]** Religious orders

In [11]:
rel_orders = attribute_to_df(
                    the_type='ordem-religiosa',
                    person_info=True,
                    more_cols=['uc-entrada','uc-saida','faculdade','titulo','colegio','naturalidade','nome-pai','padre','nome-nota'],
                    sql_echo=False)
rel_orders.drop(['nome-nota.date','nome-nota.obs','padre.date','padre.obs',
                 'ordem-religiosa.date','ordem-religiosa.obs',
                 'padre.date','padre.obs','sex',
                 'naturalidade.date','naturalidade.obs',
                 'nome-pai.date','nome-pai.obs',
                 'faculdade.date','faculdade.obs',
                'titulo.date','titulo.obs',
                'colegio.date','colegio.obs',
                'uc-entrada.date','uc-entrada.obs',
                'uc-saida.date','uc-saida.obs'],axis=1, inplace=True)
rel_orders = rel_orders[rel_orders['uc-entrada'] != '0000-00-00']
rel_orders = rel_orders[~rel_orders.index.duplicated(keep='first')]
print("Number of unique students with religious order:",len(rel_orders.index.unique()))
rel_orders.info()

Number of unique students with religious order: 3032
<class 'pandas.core.frame.DataFrame'>
Index: 3032 entries, 127987 to 319984
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             3032 non-null   object
 1   ordem-religiosa  3032 non-null   object
 2   uc-entrada       3032 non-null   object
 3   uc-saida         3032 non-null   object
 4   faculdade        3001 non-null   object
 5   titulo           2926 non-null   object
 6   colegio          123 non-null    object
 7   naturalidade     446 non-null    object
 8   nome-pai         203 non-null    object
 9   padre            1153 non-null   object
 10  nome-nota        3032 non-null   object
dtypes: object(11)
memory usage: 348.8+ KB


In [12]:
rel_orders.head()

,name,ordem-religiosa,uc-entrada,uc-saida,faculdade,titulo,colegio,naturalidade,nome-pai,padre,nome-nota
id,,,,,,,,,,,
127987,António Cabral,Ordem de São Bernardo,1665-10-15,1666-10-10,Teologia,frei,NaN,NaN,NaN,NaN,frei religioso de São Bernardo
128152,Feliciano Cabral,Ordem de São Bernardo,1699-10-01,1701-10-01,Teologia,frei,NaN,NaN,NaN,NaN,frei religioso da Ordem São Bernardo
128344,Jerónimo Cabral,Ordem de São Bernardo,1756-10-01,1756-10-01,Teologia,frei,NaN,NaN,NaN,NaN,frei monge de São Bernardo
128485,José de Nossa Senhora Cacela,Ordem de São Domingos,1751-10-01,1753-10-01,Teologia,frei,NaN,NaN,NaN,NaN,frei religioso de São Domingos
128659,Bernardo Caceres,Ordem de São Bernardo,1760-10-01,1763-10-01,Teologia,frei,NaN,NaN,NaN,NaN,frei monge de São Bernardo


In [13]:
rel_orders.sort_values(['ordem-religiosa','name']).to_csv('../inferences/name-notes/religious-orders.csv',sep=',')

In [14]:
rel_ord_count = rel_orders.groupby('ordem-religiosa')['uc-entrada'].agg(['count','min','max'])
rel_ord_count.drop(['frei','padre frei'],axis=0, inplace=True) # correct current bug, fixed for next run
rel_ord_count.sort_values('count',ascending = False, inplace=True)
rel_ord_count[rel_ord_count['count']>10]

,count,min,max
ordem-religiosa,,,
Ordem de São Bernardo,401,1569-00-00,1827-10-29
Ordem de São Bento,374,1548-01-01,1831-06-25
Ordem do Carmo,283,1536-00-00,1829-10-31
Ordem de São Francisco,267,1540-03-09,1816-10-31
Ordem de Santo Agostinho (Graça),250,1560-10-01,1780-05-30
Ordem da Santíssima Trindade,223,1551-10-00,1820-06-19
Ordem de São Domingos,202,1541-10-20,1827-10-30
Ordem de Santo Agostinho,168,1541-11-29,1829-06-03
Ordem de São Jerónimo,162,1550-03-12,1825-10-25


**[EN]** Requiring further analysis

Number of religious orders with less than 10 students

In [15]:
print("Number of cases that escape normalization:")
print(rel_ord_count[rel_ord_count['count']<=10][['count']].sum())
print("Sample:")
rel_ord_count[rel_ord_count['count']<=10].head(20)

Number of cases that escape normalization:
count    70
dtype: int64
Sample:


,count,min,max
ordem-religiosa,,,
ordem de s. francisco,7,1562-03-06,1767-10-01
Paulista,5,1752-10-01,1786-10-27
Ordem de Avis,5,1549-10-00,1626-05-13
Ordem de Santo António,4,1601-10-01,1769-10-01
Província Algarves,4,1747-10-01,1761-10-01
"padre,",2,1754-10-01,1768-01-22
Província Algarve,2,1572-10-01,1576-10-01
Ordem S. Franisco,2,1576-10-01,1791-10-22
S. José,2,1692-10-01,1801-10-03


### **[EN]** Currently expressions used to normalize the religious orders

Notes are scanned f religious order "trigger": a word that indicates that a reference is contained in the note.

Current triggers:

    ordem
    ordens
    religioso
    monge
    frei
    companhia de jesus
    colégio de jesus
    colégio dos lóios
    cónego regular
    cónego regular
    cónego regrante
    santa cruz

If one of these words is found in the note then a list of expressions are used to obtain the normalized form of the order:

In [16]:
from ucalumni.extractors import ordens

for expression in sorted(ordens.keys()):
    print(f'{expression:50} {ordens.get(expression)}')

3ª                                                 Ordem Terceira
agostinho                                          Ordem de Santo Agostinho
agostinho descalço                                 Ordem de Santo Agostinho (Descalços)
agostinhos descalços                               Ordem de Santo Agostinho (Descalços)
alcântara                                          Ordem de São Pedro de Alcântara
antónio                                            Ordem de Santo António
avis                                               Ordem de Avis
beneditino                                         Ordem de São Bento
bento                                              Ordem de São Bento
bernardo                                           Ordem de São Bernardo
carmelita                                          Ordem do Carmo
carmo                                              Ordem do Carmo
colegial do colégio de jesus                       Companhia de Jesus
colégio da graça                             

###  **[EN]** Variations

The following shows the various expressions that were mapped into a specific religious order.

Change variable "order" to examine a different case

In [17]:
order = "Ordem de São Bento"

variants = name_notes[name_notes['ordem-religiosa'] == order]['nome-nota'].unique()
print("number of expressions:",len(variants),"\nList: ",sorted(list(variants)))

number of expressions: 38 
List:  ['Frei Monge de São Bento', 'da Ordem de São Bento', 'frei da Ordem de São Bento', 'frei de Bento Lusido', 'frei de São Bento', 'frei do colégio de São Bento', 'frei monge Beneditino', 'frei monge beneditino', 'frei monge da Ordem de São Bento do Mosteiro de Sanfins', 'frei monge de São Bento', 'frei monge de São Bento do Brasil', 'frei monge de são Bento', 'frei padre da Ordem de São Bento', 'frei padre da Ordem de São Bento do Mosteiro de Pombeiro', 'frei padre religioso de S. Bento', 'frei padre religioso de São Bento', 'frei religioso da Ordem de São Bento', 'frei religioso de S. Bento', 'frei religioso de São Bento', 'frei religioso do colégio de São Bento', 'frei religioso e S. Bento', 'frei, monge beneditino', 'padre e frei da Ordem de São Bento', 'padre frei da Ordem de São Bento', 'padre frei de São Bento', 'padre frei monge de S. Bento', 'padre frei monge de São Bento', 'padre frei religioso da Ordem de S. Bento', 'padre frei religioso da Ord

## **[EN]** Place of birth



In [18]:
from timelinknb.pandas import attribute_values

df = attribute_values('naturalidade')


In [19]:

df.sort_values('count', ascending = False).head(50)

,count
value,
Lisboa,9484
Coimbra,5785
Porto,3591
Braga,1699
Évora,1160
Guimarães,1051
Lamego,1033
Viseu,1031
Aveiro,844


In [20]:
df.to_csv('../inferences/places/naturalidade-original.csv',sep=',')

In [23]:
from timelinknb.pandas import attribute_values

df = attribute_values('nome-geografico')
df.sort_values('count', ascending = False).head(50)


,count
value,
Lisboa,9606
Coimbra,6046
Porto,3814
Braga,1831
Brasil,1707
Viseu,1362
Lamego,1199
Évora,1196
Guimarães,1145


In [24]:
df.to_csv('../inferences/places/nomes-geograficos.csv',sep=',')